<div style="direction:rtl;text-align:center"><img src="https://dl.mohammadkh.ir/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/></div>
<h1><div style="direction:rtl;text-align:center">Neural Network</div></h1>

In [21]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from collections import deque
from sklearn import preprocessing
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

### Load & Preprocess data

In [20]:
main = pd.DataFrame()

currencies = ['BCH-USD', 'BTC-USD', 'ETH-USD', 'LTC-USD']

# load data and concate 
for c in currencies:
    dataset = '../../__data/crypto/' + c + '.csv'
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    df.rename(columns={'close': c + '_close', 'volume': c + '_volume'}, inplace=True)
    df.set_index('time', inplace=True)
    df = df[[c + '_close', c + '_volume']]
    if len(main) == 0:
        main = df
    else:
        main = main.join(df)

# fill nan
main.fillna(method='ffill', inplace=True)      # fill last
main.fillna(method='bfill', inplace=True)      # fill next

# creat target
## cal 3 day last 
main['future'] = main['LTC-USD_close'].shift(-3)
## cal up or down price 
def target(current, future):
    if future > current:
        return 1
    else:
        return 0
## fit fun
main['target'] = list(map(target, main['LTC-USD_close'], main['future']))


# split train and tesr
## cal len train and test
len_ = len(main.index.values)
split = sorted(main.index.values)[-int(0.1 * len_)]
## creat train and test
main_test = main[(main.index >= split)]
main_train = main[(main.index < split)]


# preprocess data
def preprocess_data(df):
    ## remove col future
    df = df.drop('future', axis=1)
    ## call diif col -->pct_change() --> diff
    for col in df.columns:
        if col != 'target':
            df[col] = df[col].pct_change()
            ## remove row  nan
            df.dropna(inplace=True)
            ## scale col
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)
    sequences = []
    ## create data time series for rnn
    prev_days = deque(maxlen=30)
    for i in df.values:
        ## split X and target
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == 30:
            sequences.append([np.array(prev_days), i[-1]])
    random.shuffle(sequences)
    
    ## balansing data
    buys = []
    sells = []
    for seq, target in sequences:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
    random.shuffle(buys)
    random.shuffle(sells)
    lower = min(len(buys), len(sells))
    buys = buys[:lower]
    sells = sells[:lower]
    sequential_data = buys + sells
    random.shuffle(sequential_data)
    
    ## criate final data
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)
## call fun
train_X, train_y = preprocess_data(main_train)
test_X, test_y = preprocess_data(main_test)

# print(main.isnull().sum())
# print(main.head(2))
# print(main_test.head(2))
print(train_X.shape)
print(train_y.shape)
print(np.unique(train_y, return_counts=True))

(71030, 30, 8)
(71030,)
(array([0., 1.]), array([35515, 35515], dtype=int64))


### create & fit model

In [29]:
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(train_X.shape[1:])))
model.add(LSTM(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 128)           70144     
                                                                 
 lstm_7 (LSTM)               (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 16)                528       
                                                                 
 dense_14 (Dense)            (None, 2)                 34        
                                                                 
Total params: 122,194
Trainable params: 122,194
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('')

In [ ]:
history = model.fit(initial_epoch=100,train_X, train_y, batch_size=100, epochs=100, validation_data=(test_X, test_y))

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong>Neural Network</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>